In [106]:
from lusidtools.jupyter_tools import toggle_code

"""
Save externally calculated metrics and use them within the Valuation Engine

Attributes
----------
"""

toggle_code("Toggle Docstring")

# Externally calculated metrics

In this notebook, we will be demonstrating how you can upload your own performance metrics to LUSID. This will allow you to use your in-house calculated P&L, Present Values and other metrics.
We will demonstrate this by first creating a portfolio, uploading holdings and then uploading our own performance metrics.

In [107]:
# Use first block to import generic non-LUSID packages
import io
import os
import pandas as pd
import numpy as np
import json
import pytz
from IPython.core.display import HTML
from datetime import datetime

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.8474.0,0.5.2581,"{'relation': 'RequestLogs', 'href': 'http://fb..."


# Setup

We must first set up some parameters and initialize the LUSID APIs. Then we will upload the instruments master, create the properties and portfolios and finally upload the holdings.

In [108]:
scope = "srs-example"

holdings_api = api_factory.build(lu.TransactionPortfoliosApi)
srs_api = api_factory.build(lu.StructuredResultDataApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
aggregation_api = api_factory.build(lu.AggregationApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)


## Instruments

In [109]:
instruments_df = pd.read_csv('data/srs_instrument_master.csv')

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instruments_df,
    mapping_required={
        "name": "Name"        
    },
    mapping_optional={},
    file_type="instruments",
    identifier_mapping={
        "ClientInternal": "ClientInternal"
    }
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,3,0,0


## Properties

In [110]:
def create_shk(shk_code, display_name):
    try:
        api_factory.build(lu.PropertyDefinitionsApi).create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=shk_code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time=None,
            )
        )
    except lu.ApiException as e:
        display(json.loads(e.body)["title"])

In [111]:
create_shk("Strategy", "Strategy")
create_shk("Sector", "Sector")
create_shk("WatchList", "WatchList")

"Error creating Property Definition 'Transaction/srs-example/Strategy' because it already exists."

"Error creating Property Definition 'Transaction/srs-example/Sector' because it already exists."

"Error creating Property Definition 'Transaction/srs-example/WatchList' because it already exists."

## Portfolio

In [112]:
portfolio_df = pd.read_csv('data/srs_portfolios.csv')

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolio_df,
    mapping_required={
        "code": "code",
        "display_name": "display_name",
        "base_currency": "base_currency"
    },
    mapping_optional={
        "created": "$2020-01-01T00:00:00+00:00"
    },
    file_type="portfolios",
    sub_holding_keys=["Strategy", "Sector", "WatchList"],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,2,0,0


## Holdings

In [113]:
holdings_df = pd.read_csv('data/srs_holdings.csv',parse_dates=['PurchaseDate', 'ValuationDate'])

In [114]:
portfolios = holdings_df.groupby(["Portfolio", "ValuationDate"])

for pf, pf_df in portfolios:
    
    tax_lot_hdlgs = pf_df.groupby(["ClientInternal", "Strategy", "Sector", "WatchList"])

    holdings_request = []    
    
    for h, tax_lots_df in tax_lot_hdlgs:
        
        tax_lots = []
        properties = {}

        for _, h in tax_lots_df.iterrows():

            tax_lots.append(
                lm.TargetTaxLotRequest(
                    units=h["Units"],
                    cost=lm.CurrencyAndAmount(amount=h["Cost"], currency=h["Currency"]),
                    price=h["Price"],
                    purchase_date=h["PurchaseDate"]
                )
            )

            properties[f"Transaction/{scope}/Strategy"] = lm.PerpetualProperty(
                key=f"Transaction/{scope}/Strategy", value=lm.PropertyValue(label_value=h["Strategy"])
            )

            properties[f"Transaction/{scope}/Sector"] = lm.PerpetualProperty(
                key=f"Transaction/{scope}/Sector", value=lm.PropertyValue(label_value=h["Sector"])
            )
            
            if h["WatchList"] == "Y":
                properties[f"Transaction/{scope}/WatchList"] = lm.PerpetualProperty(
                    key=f"Transaction/{scope}/WatchList", value=lm.PropertyValue(label_value=h["WatchList"])
                )
            else:
                properties[f"Transaction/{scope}/WatchList"] = lm.PerpetualProperty(
                    key=f"Transaction/{scope}/WatchList", value=lm.PropertyValue(label_value="N")
                )


        holdings_request.append(lm.AdjustHoldingRequest(
            instrument_identifiers={ "Instrument/default/ClientInternal": h["ClientInternal"] },
            tax_lots=tax_lots,
            sub_holding_keys=properties
        ))
        
    holdings_api.set_holdings(scope=scope, code=pf[0], effective_at=h["ValuationDate"], adjust_holding_request=holdings_request)


# Structured Results Store

Now that we have our portfolios set up, we will demonstrate how you can upsert your own valuation data using the structured results store. We are using the Market Value (MV) and Performance (GainLoss) numbers from our holdings dataset we used earlier in the notebook.

In [115]:
srs_source_df = holdings_df[["ValuationDate", "Portfolio", "Strategy", "Sector", "WatchList", "ClientInternal", "Currency", "MV", "GainLoss"]]
srs_source_df

,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss
0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00
1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00
2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00
3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00
4,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,VOD LN,GBP,nan,nan
5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00
6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00
7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00
8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00


To effectively use the structured results store, we must know the scope and Lusid generated Instrument Ids for each instrument. We can utilize the instruments API to collect this data.

In [116]:
instruments = instruments_api.get_instruments(identifier_type="ClientInternal", request_body=list(srs_source_df["ClientInternal"].unique()))

In [117]:
ci_to_luid = {
    ci: inst.identifiers["LusidInstrumentId"]
    for ci, inst in instruments.values.items()
}

srs_source_df["Scope"] = scope
srs_source_df["Luid"] = srs_source_df.apply(lambda x: ci_to_luid[x["ClientInternal"]], axis=1)
display(srs_source_df)

C:\Users\LORENZ~1\AppData\Local\Temp/ipykernel_13324/3837856390.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srs_source_df["Scope"] = scope
C:\Users\LORENZ~1\AppData\Local\Temp/ipykernel_13324/3837856390.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srs_source_df["Luid"] = srs_source_df.apply(lambda x: ci_to_luid[x["ClientInternal"]], axis=1)


,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00,srs-example,LUID_01JIIHOO
1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00,srs-example,LUID_SQYHWQH6
2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00,srs-example,LUID_01JIIHOO
3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00,srs-example,LUID_01JIIHOO
4,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,VOD LN,GBP,nan,nan,srs-example,LUID_OYAGXA0M
5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00,srs-example,LUID_01JIIHOO
6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00,srs-example,LUID_SQYHWQH6
7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00,srs-example,LUID_01JIIHOO
8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00,srs-example,LUID_01JIIHOO


## Data Map

Now that we have our dataset with our in-house valuations (the MV and GainLoss columns above), we can map these and upload them to Lusid.

In [118]:
srs_data_map = lm.DataMapping(
    data_definitions=[

        # composite key         
        lm.DataDefinition(address="UnitResult/Portfolio/Id", name="Portfolio", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Portfolio/Scope", name="Scope", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/Strategy", name="Strategy", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/Sector", name="Sector", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address=f"UnitResult/Transaction/{scope}/WatchList", name="WatchList", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Instrument/default/LusidInstrumentId", name="Luid", data_type="string", key_type="PartOfUnique"),
        lm.DataDefinition(address="UnitResult/Holding/default/Currency", name="Currency", data_type="string", key_type="PartOfUnique"),

        # holding values         
        lm.DataDefinition(address="UnitResult/MV", name="MV", data_type="decimal", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/GainLoss", name="GainLoss", data_type="decimal", key_type="Leaf"),
    ]
)

srs_data_map_key = lm.DataMapKey(version="0.1.8", code="market-valuation-map")

try:    
    srs_api.create_data_map(
        scope=scope, 
        request_body={
            "market-valuation-map": lm.CreateDataMapRequest(
                id=srs_data_map_key,
                data=srs_data_map
            )
        }
    )
except lu.ApiException as e:
    display(json.loads(e.body))

## Upsert Data

In [119]:
srs_ids = []

for effective_at, srs_df in srs_source_df.groupby("ValuationDate"):

    srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="MarketValuation",
        effective_at=effective_at,
        result_type = "UnitResult/Holding"
    )
    
    srs_ids.append(srs_data_id)
    
    # filter out rows without any MV and GainLoss values
    srs_df = srs_df[~srs_df[["MV", "GainLoss"]].isna().all(1)]
    s = io.StringIO()
    srs_df.to_csv(s)
    
    srs_data = lm.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Market valuations",
        data_map_key=srs_data_map_key,
        document=s.getvalue()        
    )
    
    srs_api.upsert_structured_result_data(
        scope=scope, 
        request_body={ 
            "data": lm.UpsertStructuredResultDataRequest(
                id=srs_data_id, 
                data=srs_data
            )
        }
    )

## Read from SRS

We check if everything has been uploaded correctly by retrieving the previously uploaded data.
Here we can see that our upload was successful and the structured results store contains our data.

In [120]:
for sid in srs_ids:
    
    key = f"{sid.code}-{effective_at}"
    
    values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: sid
        }
    )
    
    s = io.StringIO(values.values[key].document)
    values_df = pd.read_csv(s)
    
    display(values_df)

,Unnamed: 0,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00,srs-example,LUID_01JIIHOO
1,1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00,srs-example,LUID_SQYHWQH6
2,2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00,srs-example,LUID_01JIIHOO
3,3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00,srs-example,LUID_01JIIHOO


,Unnamed: 0,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00,srs-example,LUID_01JIIHOO
1,6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00,srs-example,LUID_SQYHWQH6
2,7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00,srs-example,LUID_01JIIHOO
3,8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00,srs-example,LUID_01JIIHOO


# Valuation Engine
We will now run the valuation engine to see if it populates our structured results store data instead of calculating the performance data by itself.

## Recipe

In [121]:
recipe_code = "MarketValuation"

recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    pricing=lm.PricingContext(
        result_data_rules=[
            lm.ResultDataKeyRule(
                resource_key="UnitResult/*",
                supplier="Client",
                data_scope=scope,
                document_code="MarketValuation",
                quote_interval="1D"
            )
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation=True,
            allow_any_instruments_with_sec_uid_to_price_off_lookup=False
        )                              
    )
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = lm.UpsertRecipeRequest(
        configuration_recipe = recipe
    )
)

In [122]:
def run_valuation(portfolio_code, effective_at):
    
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_at.isoformat()),
        metrics=[
            lm.AggregateSpec("Portfolio/Scope", "Value"),
            lm.AggregateSpec("Portfolio/Id", "Value"),
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/Strategy", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/Sector", "Value"),
            lm.AggregateSpec(f"Transaction/{scope}/WatchList", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("UnitResult/MV", "Value"),
            lm.AggregateSpec("UnitResult/GainLoss", "Value"),
        ]
    )
    
    valuation_result = aggregation_api.get_valuation(valuation_request=valuation_request)
    return pd.DataFrame(valuation_result.data)

Now that we have specified the results store data in our recipe and valuation request, we will now run these valuation requests to check if the data returned is indeed the data we uploaded.
We will run these for our 3 dates which we specify below.

In [123]:
day1 = datetime(2021, 8, 1, tzinfo=pytz.utc)
day2 = datetime(2021, 8, 2, tzinfo=pytz.utc)
day3 = datetime(2021, 8, 3, tzinfo=pytz.utc)

Let's first have a look at the EquityA portfolio. Recall from our source file contained in the dataframe "srs_source_df" that we have uploaded the below MV and GainLoss values:

In [124]:
srs_source_df[srs_source_df['Portfolio'] == "UKEquityA"]

,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
0,2021-08-01 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3600.00,400.00,srs-example,LUID_01JIIHOO
1,2021-08-01 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7200.00,600.00,srs-example,LUID_SQYHWQH6
2,2021-08-01 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6300.00,700.00,srs-example,LUID_01JIIHOO
5,2021-08-02 00:00:00+00:00,UKEquityA,Growth,Financial,N,BARC LN,GBP,3700.00,500.00,srs-example,LUID_01JIIHOO
6,2021-08-02 00:00:00+00:00,UKEquityA,Value,Consumer,Y,TSCO LN,GBP,7500.00,900.00,srs-example,LUID_SQYHWQH6
7,2021-08-02 00:00:00+00:00,UKEquityA,Value,Financial,N,BARC LN,GBP,6475.00,875.00,srs-example,LUID_01JIIHOO


We can now check if we are receiving these same values for UKEquityA from our valuation results:

In [125]:
run_valuation("UKEquityA", day1)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityA,Barclays,Growth,Financial,N,2000.00,3600.00,400.00
1,srs-example,UKEquityA,Barclays,Value,Financial,N,3500.00,6300.00,700.00
2,srs-example,UKEquityA,Tesco,Value,Consumer,Y,3000.00,7200.00,600.00


In [126]:
run_valuation("UKEquityA", day2)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityA,Barclays,Growth,Financial,N,2000.00,3700.00,500.00
1,srs-example,UKEquityA,Barclays,Value,Financial,N,3500.00,6475.00,875.00
2,srs-example,UKEquityA,Tesco,Value,Consumer,Y,3000.00,7500.00,900.00


We can see that these values are the same as from the file we uploaded. We now do the same for the UKEquityB portfolio:

In [127]:
srs_source_df[srs_source_df['Portfolio'] == "UKEquityB"]

,ValuationDate,Portfolio,Strategy,Sector,WatchList,ClientInternal,Currency,MV,GainLoss,Scope,Luid
3,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9000.00,-500.00,srs-example,LUID_01JIIHOO
4,2021-08-01 00:00:00+00:00,UKEquityB,Growth,Financial,N,VOD LN,GBP,nan,nan,srs-example,LUID_OYAGXA0M
8,2021-08-02 00:00:00+00:00,UKEquityB,Growth,Financial,N,BARC LN,GBP,9250.00,-250.00,srs-example,LUID_01JIIHOO


In [128]:
run_valuation("UKEquityB", day1)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss,Aggregation/Errors
0,srs-example,UKEquityB,Barclays,Growth,Financial,N,5000.00,9000.00,-500.00,[]
1,srs-example,UKEquityB,Vodafone Group,Growth,Financial,N,2000.00,nan,nan,[Failed to resolve result data item [UnitResul...


In [129]:
run_valuation("UKEquityB", day2)

,Portfolio/Scope,Portfolio/Id,Instrument/default/Name,Transaction/srs-example/Strategy,Transaction/srs-example/Sector,Transaction/srs-example/WatchList,Holding/default/Units,UnitResult/MV,UnitResult/GainLoss
0,srs-example,UKEquityB,Barclays,Growth,Financial,N,5000.00,9250.00,-250.00
